In [1]:
import pandas as pd
import os
import random
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [10]:
from sqlalchemy import create_engine
engine = create_engine("mysql+pymysql://{user}:{pw}@{host}:{port}/{db}?charset=utf8mb4"
                       .format(user="openalex_ym",
                               pw="openalex.ym",
                               host="172.18.18.167",
                               port=3306,
                               db="openalex2022"),
                       )

In [22]:
Mtoship = pd.read_csv('pid_mag_openalex.csv')

In [23]:
len(Mtoship)

499386

In [24]:
Mtoship = Mtoship[~Mtoship.author_id.isnull()]

In [25]:
len(Mtoship)

479837

In [26]:
Mtoship.head(5)

,PID,MAGAuthorID,author_id
0,1,2.171828e+09,A2171827615
1,2,2.082173e+09,A2082172561
2,3,2.110166e+09,A2110165986
3,4,2.183264e+09,A2183263527
4,5,2.156587e+09,A2156586888


In [27]:
auid = list(set(Mtoship.author_id))
print(len(auid))

472187


In [28]:
author_groups = []
interval = 5000
start_point = 0
while True:
    end_point = start_point + interval
    try:
        temp = auid[start_point:end_point]
        author_groups.append(temp)
    except:
        temp = auid[start_point:]
        author_groups.append(temp)
        
    start_point = end_point
    if end_point>len(auid):
        break

In [29]:
l = len(author_groups)
print(l)

95


In [30]:
l = len(author_groups)
for i in range(l):
    print(i)
    temp =  author_groups[i]
    sql_query = """select works_authorships.author_id, works_authorships.work_id

                    from works_authorships 

                    where works_authorships.author_id in {}
                    """.format(tuple(temp))


    data = pd.read_sql(sql_query, con=engine)
    # merge data
    if i==0:
        tp = data
    else:
        tp = pd.concat((tp,data))

data = tp

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94


In [31]:
data.to_csv('all_AFT_mentee_pubs.csv',index=0)

#### find paper_year

In [32]:
paper = list(set(data.work_id))
print(len(paper))

18568113


In [33]:
paper_groups = []
interval = 5000
start_point = 0
while True:
    end_point = start_point + interval
    try:
        temp = paper[start_point:end_point]
        paper_groups.append(temp)
    except:
        temp = paper[start_point:]
        paper_groups.append(temp)
        
    start_point = end_point
    if end_point>len(paper):
        break

In [34]:
l = len(paper_groups)
print(l)
for i in range(l):
    temp =  paper_groups[i]
    sql_query = """select works.id, works.publication_year

                    from works 

                    where works.id in {}
                    """.format(tuple(temp))


    data = pd.read_sql(sql_query, con=engine)
    # merge data
    if i==0:
        tp = data
    else:
        tp = pd.concat((tp,data))
    if i%10==0:
        print(i)

Data = tp

3714
0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
1620
1630
1640
1650
1660
1670
1680
1690
1700
1710
1720
1730
1740
1750
1760
1770
1780
1790
1800
1810
1820
1830
1840
1850
1860
1870
1880
1890
1900
1910
1920
1930
1940
1950
1960
1970
1980
1990
2000
2010
2020
2030
2040
2050
2060
2070
2080
2090
2100
2110
2120
2130
2140
2150
2160
2170
2180
2190
2200
2

In [35]:
Data = Data.rename(columns={'id':'work_id'})

In [36]:
data = pd.read_csv('all_AFT_mentee_pubs.csv')

In [37]:
data = data.merge(Data,on='work_id',how='left')

In [39]:
# data.to_csv('all_AFT_mentee_pubs_year.csv',index=0)

##### fing career_span

In [40]:
author_year = data[['author_id','publication_year']].drop_duplicates()

In [41]:
idx = author_year.groupby(['author_id'])['publication_year'].transform(min) == author_year['publication_year'] #choose the minimum publication_year 
first_pub_year = author_year[idx]
idx = author_year.groupby(['author_id'])['publication_year'].transform(max) == author_year['publication_year'] #choose the mamum publication_year 
last_pub_year = author_year[idx]

first_pub_year = first_pub_year.merge(last_pub_year,on='author_id',how='left')
first_pub_year = first_pub_year.rename(columns={'publication_year_x':'first_pub_year','publication_year_y':'last_pub_year'})
first_pub_year['career_len'] = first_pub_year.last_pub_year - first_pub_year.first_pub_year

In [42]:
first_pub_year.to_csv('all_AFT_mentee_career_span.csv',index=0)